In [ ]:
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Load and Process Documents
doc_folder = "personal_docs"  # Your folder
documents = []
metadata = []

def load_documents(folder):
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        if filename.endswith((".txt", ".py", ".pdf")):
            try:
                if filename.endswith(".pdf"):
                    reader = PdfReader(filepath)
                    text = "".join(page.extract_text() for page in reader.pages if page.extract_text())
                    if not text.strip():
                        print(f"Warning: No text extracted from {filename}")
                        continue
                else:
                    with open(filepath, "r", encoding="utf-8") as f:
                        text = f.read()
                documents.append(text)
                metadata.append({"source": filename, "type": "pdf" if filename.endswith(".pdf") else "text"})
            except Exception as e:
                print(f"Skipping {filename}: {str(e)}")

load_documents(os.path.expanduser(doc_folder))

# Split documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
doc_chunks = text_splitter.create_documents(documents, metadatas=metadata)
documents = [doc.page_content for doc in doc_chunks]
chunk_metadata = [doc.metadata for doc in doc_chunks]

# 2. Set Up Retriever
retriever = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = retriever.encode(documents, batch_size=32, show_progress_bar=True)
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

def retrieve(query, k=2):  # Reduced k to limit context
    query_embedding = retriever.encode([query])
    distances, indices = index.search(query_embedding, k)
    return [(documents[idx], chunk_metadata[idx]) for idx in indices[0]]

# 3. Set Up Generator
model_name = "G:\My Drive\llama-3.2-3b-instruct"  # Your model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

def generate_response(query, retrieved_docs):
    context = "\n".join([f"[{meta['source']}]: {doc[:200]}..." for doc, meta in retrieved_docs])  # Limit doc length
    prompt = (
        f"You are my personal AI assistant, specialized in my research on Progressive Neural Networks (PNNs) and LLaMA. "
        f"Using only the provided documents, explain my work concisely and technically, citing sources like '[filename]'. "
        f"If the query is about a paper, summarize its main contribution and key methods. "
        f"Documents:\n{context}\n\nQuery: {query}\n\nAnswer:\n"
    )
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=250, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Robust parsing
    answer = response.split("Answer:")[-1].strip() if "Answer:" in response else response.strip()
    # Fallback if parsing fails
    if not answer or answer.startswith("You are my personal AI assistant"):
        answer = "Unable to generate a specific answer. Please check document content or query phrasing."
    return answer

# 4. RAG Pipeline
def personal_rag(query, k=2):
    retrieved_docs = retrieve(query, k)
    response = generate_response(query, retrieved_docs)
    return response, retrieved_docs

# 5. Test the Pipeline
query = "Explain about the authors of this paper"
answer, sources = personal_rag(query)
print(f"Query: {query}\nAnswer: {answer}\nSources: {[meta['source'] for _, meta in sources]}")

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
import gc # Import the garbage collector module


def process_query(query):
  """Processes the query and returns the response."""

  # Load and split document
  loader = TextLoader("Latest_news.txt", encoding="utf-8")
  docs = loader.load()
  splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
  chunks = splitter.split_documents(docs)

  # Create vector database
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
  vector_db = FAISS.from_documents(chunks, embedding_model)

  # Similarity search
  retrieved_docs = vector_db.similarity_search(query, k=3)
  context = "\n".join([doc.page_content for doc in retrieved_docs])

  # Generate response using LLM
  llm = pipeline("text-generation", model="G:\My Drive\llama-3.2-3b-instruct")
  prompt = f"Answer the question based on the context:\n{context}\n\nQuestion: {query}\nAnswer:"
  response = llm(prompt, max_new_tokens=300)[0]["generated_text"]

  # Clean up memory
  del loader, docs, splitter, chunks, embedding_model, vector_db, retrieved_docs, context, llm, prompt
  gc.collect()  # Explicitly trigger garbage collection

  return response

# Example usage
query = "Thanks for your information"
response = process_query(query)
print(response)